## Introduction:

This notebook is written in light of the below link:

https://colab.research.google.com/github/RasaHQ/rasa_core/blob/master/getting_started.ipynb#scrollTo=zQxY6m6kvlj3

In [ ]:
# !pip install -U git+https://github.com/rasahq/rasa_core.git

In [ ]:
# !pip install rasa_core;

In [ ]:
import logging
from rasa_core import training
from rasa_core.actions import Action
from rasa_core.agent import Agent
from rasa_core.domain import Domain
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.featurizers import MaxHistoryTrackerFeaturizer, BinarySingleStateFeaturizer

## for NLU!
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer


## visualize the stories .. 
from rasa_core.agent import Agent
from IPython.display import IFrame


import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

In [ ]:
agent = Agent('domain.yml', policies=[MemoizationPolicy(max_history=4), KerasPolicy()])
training_data = agent.load_data('stories.md')
agent.train(
        training_data,
        validation_split=0.30,
        epoch=200, 
        num_features = 5,
        max_dialogue_length = 10,
        num_stories = 100,
        augmentation_factor=1000
    
    ## featurizer
#https://rasa.com/docs/core/0.10.4/policies/
#     interpreter=interpreter,
#     generator=generator,
#     tracker_store=tracker_store,
#     action_endpoint=action_endpoint
)

In [ ]:
agent.persist('models/dialogue')

In [ ]:
agent = Agent.load('models/dialogue')
agent

## Talking to the bot .. 

No NLU yet!

In [ ]:
# print("Your bot is ready to talk! Type your messages here or send 'stop'")
# while True:
#     a = input()
#     if a == 'stop':
#         break
#     responses = agent.handle_text(a)
#     for response in responses:
#         print(response["text"])

## Adding NLU: 

To allow for more intelligence .. 

In [ ]:
# !pip install git+git://github.com/pygraphviz/pygraphviz.git;

In [ ]:
# !pip install rasa_nlu[spacy];
# !python -m spacy download en;

In [ ]:
training_data = load_data('./nlu_data.json')
training_data

In [ ]:
pipeline = [{"name": "nlp_spacy"}, 
            {"name": "tokenizer_spacy"}, 
            {"name": "intent_featurizer_spacy"},  
            {"name": "intent_classifier_sklearn"}]

In [ ]:
trainer = Trainer(RasaNLUModelConfig({"pipeline": pipeline}))
trainer

In [ ]:
interpreter = trainer.train(training_data)
interpreter

In [ ]:
model_directory = trainer.persist('./projects/default/')

In [ ]:
agent = Agent.load('models/dialogue', interpreter=model_directory)
agent

## Talking to the Bot:

NLU is added!

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_text(a)
    for response in responses:
        print(response["text"])

## Visualizing the stories:

In [17]:
agent = Agent.load('models/dialogue')
agent.visualize("stories.md", "story_graph.html", max_history=2)

Processed Story Blocks: 100%|█████████████████████████████████████████| 102/102 [00:00<00:00, 482.42it/s, # trackers=1]


In [18]:
IFrame(src="./story_graph.html", width=1000, height=800)

End of the first tutorial!